In [2]:
import pandas as pd

In [5]:
data=pd.read_csv("./wdbc.data", header=None)
cols=['id','class']
feature_list=["radius","texture","perimeter","area","smoothness","compactness","concavity","concave_points","symmetry","fractal_dim"]
for first in ['mean_','se_','worst_']:
    for item in feature_list:
        cols.append(first+item)
data.columns=cols
data=data.drop('id',axis=1)

# 전처리


## 1) Categorical data

In [6]:
from sklearn import preprocessing

label_encoder=preprocessing.LabelEncoder()
label_encoder.fit(data['class'])
data['class_']=label_encoder.transform(data['class'])
data[['class_','class']]

,class_,class
0,1,M
1,1,M
2,1,M
3,1,M
4,1,M
...,...,...
564,1,M
565,1,M
566,1,M
567,1,M


- 이후 원래대로 돌릴 경우

In [7]:
data['class_2']=label_encoder.inverse_transform(data['class_'])
data[['class_','class','class_2']]

,class_,class,class_2
0,1,M,M
1,1,M,M
2,1,M,M
3,1,M,M
4,1,M,M
...,...,...,...
564,1,M,M
565,1,M,M
566,1,M,M
567,1,M,M


[RandomForest 하이퍼 파라미터 조정(GridSearch)](https://injo.tistory.com/30)

[GridSearchCV](https://2-chae.github.io/category/1.ai/29)

# 1. GridSearchCV를 통한 랜덤포레스트의 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# 2. 위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 

   # 테스트 세트 데이터에서 예측 성능을 측정

In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 12,
                                min_samples_leaf = 8,
                                min_samples_split = 8,
                                random_state = 0,
                                n_jobs = -1)
rf_clf1.fit(X_train, y_train)
pred = rf_clf1.predict(X_test)
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

# 3. Random Forest의 각 피처의 중요도 시각화 : featureimportances

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index = X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Top 20 Feature Importances')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

# 4. Confusion Matrix

In [ ]:
conf_mat = confusion_matrix(labels, predictions, normalize='true')

disp = plot_confusion_matrix(classifier, X_test, y_test,
display_labels=class_names, cmap=plt.cm.Blues, normalize='true')